In [8]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

In [9]:
import copy
import numpy as np
import time

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Dataset

In [17]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
batch_size = 4

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform)
trainset = torch.utils.data.Subset(trainset, torch.arange(5000))
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True, num_workers = 2)
testset  = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform)
testset  = torch.utils.data.Subset(trainset, torch.arange(1000))
testloader  = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False, num_workers = 2)

dataloaders = {'train' : trainloader, 'val':testloader}

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
model = GoogLeNet(10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
Tr = Trainer('classification', googlenet_training_routine, googlenet_validation_routine)
model, training_info = Tr.train(model, dataloaders, criterion, optimizer, num_epochs=10)